# Preprocessing: prescriptions Table

In [1]:
import pickle
import os
os.chdir('../../')
from utils.hosp_preprocess_util import *    # module of preprocessing functions

### Reading in Prescriptions Data

In [ ]:
mimic4_path = "./mimic-iv-1.0/hosp/prescriptions.csv.gz"
med = pd.read_csv(mimic4_path, compression='gzip', usecols=['subject_id', 'hadm_id', 'drug', 'starttime', 'ndc'])
med.head()

,subject_id,hadm_id,starttime,drug,ndc
0,17868682,22726960,2160-01-07 08:00:00,BuPROPion (Sustained Release),591083960.0
1,17868682,22726960,2160-01-07 08:00:00,BuPROPion (Sustained Release),591083960.0
2,17868682,22726960,2160-01-07 10:00:00,Aspirin,904404073.0
3,17868682,22726960,2160-01-07 14:00:00,1/2 NS,338004304.0
4,17868682,22726960,2160-01-07 15:00:00,Pneumococcal Vac Polyvalent,6494300.0


In [ ]:
med.shape

(17008053, 5)

In [ ]:
print("Number of unique entires in \'drug\' col:                                    ", med.drug.dropna().nunique())
print("Number of unique entires in \'drug\' col after applying lower() and strip(): ", med.drug.dropna().apply(lambda x: x.lower().strip()).nunique())

Number of unique entires in 'drug' col:                                     10254
Number of unique entires in 'drug' col after applying lower() and strip():  8357


In [ ]:
# Normalize drug strings and remove potential duplicates

med.drug = med.drug.fillna("").astype(str)
med.drug = med.drug.apply(lambda x: x.lower().strip().replace(" ", "_") if not "" else "")
# med = med.drop_duplicates(subset=['hadm_id', 'drug'])

med.shape

(17008053, 5)

### Filtering Based on NDC Codes

In [ ]:
# Convert any nan values to a dummy value
med.ndc = med.ndc.fillna(-1)

# Ensures the decimal is removed from the ndc col
med.ndc = med.ndc.astype("Int64")

In [ ]:
# The NDC codes in the prescription dataset is the 11-digit NDC code, although codes are missing
# their leading 0's because the column was interpreted as a float then integer; this function restores
# the leading 0's, then obtains only the PRODUCT and MANUFACTUERER parts of the NDC code (first 9 digits)
def to_str(ndc):
    if ndc < 0:         # dummy values are < 0
        return np.nan
    ndc = str(ndc)
    return (("0"*(11 - len(ndc))) + ndc)[0:-2]

# The mapping table is ALSO incorrectly formatted for 11 digit NDC codes. An 11 digit NDC is in the
# form of xxxxx-xxxx-xx for manufactuerer-product-dosage. The hyphens are in the correct spots, but
# the number of digits within each section may not be 5-4-2, in which case we add leading 0's to each
# to restore the 11 digit format. However, we only take the 5-4 sections, just like the to_str function
def format_ndc_table(ndc):
    parts = ndc.split("-")
    return ("0"*(5 - len(parts[0])) + parts[0]) + ("0"*(4 - len(parts[1])) + parts[1])

In [ ]:
# Read in NDC mapping table
ndc_map = read_ndc_mapping('./utils/mappings/ndc_product.txt')[['productndc', 'nonproprietaryname', 'pharm_classes']]

### The full_map code is optional if you want to use the excluded and unfinished NDC tables ###

# ndc_map_ex = read_ndc_mapping('./utils/mappings/ndc_products_excluded.txt')
# ndc_map_unf = read_ndc_mapping('./utils/mappings/ndc_unfinished_product.txt')

# full_map = pd.concat([ndc_map[['productndc', 'nonproprietaryname', 'pharm_classes']], ndc_map_unf[['productndc', 'nonproprietaryname', 'pharm_classes']]], axis=0)
# full_map.drop_duplicates(inplace=True)
# full_map = ndc_map[['productndc', 'nonproprietaryname', 'pharm_classes']]
# full_map.head()

In [ ]:
# Normalize the NDC codes in the mapping table so that they can be merged
ndc_map['new_ndc'] = ndc_map.productndc.apply(format_ndc_table)
ndc_map.drop_duplicates(subset=['new_ndc', 'nonproprietaryname'], inplace=True)
med['new_ndc'] = med.ndc.apply(to_str)    

In [ ]:
# Check how many NDC codes exist before merging with mapping table
print("# of Adjusted 11-Digit NDC codes in prescriptions data:                  ", med.new_ndc.nunique())

# Left join the med dataset to the mapping information
med = med.merge(ndc_map, how='inner', left_on='new_ndc', right_on='new_ndc')

print("# of Adjusted 11-Digit NDC codes after mergining with NDC mapping table: ", med.new_ndc.nunique())

# of Adjusted 11-Digit NDC codes in prescriptions data:                   5527
# of Adjusted 11-Digit NDC codes after mergining with NDC mapping table:  2991


In [ ]:
# Mapping table maps nonproprietary drug names to NDC codes, check the unique counts per patient
med_counts = med.drop_duplicates(subset=['nonproprietaryname', 'subject_id']).nonproprietaryname.value_counts()
med_counts[:15]

heparin sodium                                                              120190
sodium chloride                                                             117496
dextrose monohydrate                                                         84219
sodium chloride, potassium chloride, sodium lactate and calcium chloride     79494
oxycodone hydrochloride                                                      78711
bisacodyl                                                                    71802
acetaminophen                                                                68826
magnesium sulfate in water                                                   63403
potassium chloride                                                           63317
phytonadione                                                                 58419
docusate sodium                                                              56687
hydromorphone hydrochloride                                                  53493
onda

### Filtering Based on Established Pharmacologic Class (EPC)

EPCs are another label in the NDC mapping table that categorizes drugs, which further reduces the number of classes in the dataset

In [ ]:
# In NDC mapping table, the pharm_class col is structured as a text string, separating different pharm classes from eachother
# This can be [PE], [EPC], and others, but we're interested in EPC. Luckily, between each commas, it states if a phrase is [EPC]
# So, we just string split by commas and keep phrases containing "[EPC]"
def get_EPC(s):
    """Gets the Established Pharmacologic Class (EPC) from the mapping table"""
    if type(s) != str:
        return np.nan
    words = s.split(",")
    return [x for x in words if "[EPC]" in x]

In [ ]:
# Function generates a list of EPCs, as a drug can have multiple EPCs
med['EPC'] = med.pharm_classes.apply(get_EPC)

In [ ]:
# Observe modified dataset. When One hot encoding and pivoting, you need to remove NaN and duplicates from the drug/nonpropietaryname/EPC
# Duplicates should be removed using drop_duplicates(subset=[hadm_id, target_col])
med

,subject_id,hadm_id,starttime,drug,ndc,new_ndc,productndc,nonproprietaryname,pharm_classes,EPC
0,17868682,22726960,2160-01-07 10:00:00,aspirin,904404073,009044040,0904-4040,aspirin,NaN,NaN
1,17067646,20845642,2159-02-23 10:00:00,aspirin,904404073,009044040,0904-4040,aspirin,NaN,NaN
2,17067646,25358552,2159-08-08 10:00:00,aspirin,904404073,009044040,0904-4040,aspirin,NaN,NaN
3,13359788,27483342,2143-11-22 19:00:00,aspirin,904404073,009044040,0904-4040,aspirin,NaN,NaN
4,15346117,20604717,2195-01-21 10:00:00,aspirin,904404073,009044040,0904-4040,aspirin,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9263468,10760672,29128660,2129-06-07 08:00:00,deferoxamine_(dfo),409233725,004092337,0409-2337,deferoxamine mesylate,"Iron Chelating Activity [MoA],Iron Chelator [EPC]",[Iron Chelator [EPC]]
9263469,10760672,29128660,2129-06-08 08:00:00,deferoxamine_(dfo),409233725,004092337,0409-2337,deferoxamine mesylate,"Iron Chelating Activity [MoA],Iron Chelator [EPC]",[Iron Chelator [EPC]]
9263470,12744652,29642134,2130-12-13 15:00:00,deferoxamine_(dfo),409233725,004092337,0409-2337,deferoxamine mesylate,"Iron Chelating Activity [MoA],Iron Chelator [EPC]",[Iron Chelator [EPC]]
9263471,15350929,27196331,2128-07-05 08:00:00,deferoxamine_(dfo),409233725,004092337,0409-2337,deferoxamine mesylate,"Iron Chelating Activity [MoA],Iron Chelator [EPC]",[Iron Chelator [EPC]]


In [ ]:
# Most common EPCs in this chunk of the dataset
# epc_med = pd.DataFrame(output, columns=mlb.classes_)
# epc_med.sum().sort_values(ascending=False).head(20)

### Saving the Modified Prescriptions Table

Now, we save the modified dataset with our collumns of interest

In [ ]:
med[['subject_id', 'hadm_id', 'starttime',	'drug',	'nonproprietaryname', 'EPC']].to_csv('./data/meds/prescriptions_full.csv.gz', compression='gzip', index=False)

### Getting the Modified Prescriptions table into long format with timedelta

In [ ]:
# The timestamp_cohort_data turns the dataset into the long-format for generating the final dataset
# Three datasets are generating for three separate feature columns
for col, idx in zip(["drug", "nonproprietaryname"],[3, 4]):
    use_mlb = False
    if col == "EPC":
        use_mlb = True
    df = timestamp_cohort_data('./data/meds/prescriptions_full.csv.gz', './data/cohort.gzip', dtypes=None, time_col='starttime', anchor_col='base_anchor_year',usecols=[0, 1, 2, idx])
    df[['subject_id', 'hadm_id', col, 'starttime']].dropna().to_csv("./data/long_format/meds/preproc_med_" + col +".csv.gz", compression='gzip', index=False)

    # pivot_df = pivot_cohort(df, prefix="med_", target_col=col, values='values', use_mlb=use_mlb, max_features=100)

    # pivot_df.fillna(0).astype(int).to_pickle('./data/meds/pivot_' + col + '.gzip', compression='gzip')

In [ ]:
# Check that data was generated properly
pd.read_pickle('./data/long_format/meds/long_med_EPC.gzip', compression='gzip')

## OneHotEncoding and Pivoting

In [ ]:
# ohe_med = timestamp_cohort_data('./data/prescriptions_full.csv.gz', './data/cohort.gzip', dtypes=None, time_col='starttime', usecols=[0, 1, 2])
# pivot_med = pivot_cohort(ohe_med, prefix="med_", target_col='drug', values='values', ohe=True)

# ohe_med_nonprop = timestamp_cohort_data('./data/prescriptions_full.csv.gz', './data/cohort.gzip', dtypes=None, time_col='starttime', usecols=[0, 1, 3])
# pivot_med_nonprop = pivot_cohort(ohe_med_nonprop, prefix="med_", target_col='nonproprietaryname', values='values', ohe=True)

# ohe_med_EPC = timestamp_cohort_data('./data/prescriptions_full.csv.gz', './data/cohort.gzip', dtypes=None, time_col='starttime', usecols=[0, 1, 4])
# pivot_med_EPC = pivot_cohort(ohe_med_EPC, prefix="med_", target_col='EPC', values='values', use_mlb=True)

C:\Users\Brennan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
